In [1]:
import requests
import os

API_KEY = os.getenv("API_KEY")

SPORT = 'soccer_epl' 
REGIONS = 'uk' 
MARKETS = 'h2h' 
ODDS_FORMAT = 'decimal' 
DATE_FORMAT = 'iso' 
BOOKIES = "williamhill,paddypower,unibet_uk,skybet,marathonbet"


In [2]:
odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
        'bookmakers': BOOKIES
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()

    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])


Remaining requests 491
Used requests 9


In [3]:
import pandas as pd
from collections import defaultdict

df = pd.json_normalize(
    odds_json,
    record_path=['bookmakers','markets','outcomes'],
    meta=['id','sport_key','sport_title','commence_time','home_team','away_team',
            ['bookmakers','title'], ['bookmakers','last_update'],
            ['bookmakers','markets','last_update']],
    errors='ignore'
)

df = (df.rename(columns={
        'id':'event_id',
        'bookmakers.title':'bookmaker',
        'bookmakers.last_update':'bookmaker_last_update',
        'bookmakers.markets.last_update':'market_last_update',
        'name':'outcome_name',
        'price':'odds_decimal'
    })
    [['event_id','commence_time','home_team','away_team',
        'bookmaker','bookmaker_last_update',
        'outcome_name','odds_decimal']]
)

print(df.head(10))


                           event_id         commence_time  home_team  \
0  2f666526770611c3cd61ee116724fe6b  2025-10-25T18:59:00Z  Brentford   
1  2f666526770611c3cd61ee116724fe6b  2025-10-25T18:59:00Z  Brentford   
2  2f666526770611c3cd61ee116724fe6b  2025-10-25T18:59:00Z  Brentford   
3  2f666526770611c3cd61ee116724fe6b  2025-10-25T18:59:00Z  Brentford   
4  2f666526770611c3cd61ee116724fe6b  2025-10-25T18:59:00Z  Brentford   
5  2f666526770611c3cd61ee116724fe6b  2025-10-25T18:59:00Z  Brentford   
6  2f666526770611c3cd61ee116724fe6b  2025-10-25T18:59:00Z  Brentford   
7  2f666526770611c3cd61ee116724fe6b  2025-10-25T18:59:00Z  Brentford   
8  2f666526770611c3cd61ee116724fe6b  2025-10-25T18:59:00Z  Brentford   
9  e66635d9df0b4ed520d8c6b9847f56aa  2025-10-26T14:00:00Z    Arsenal   

        away_team    bookmaker bookmaker_last_update outcome_name  \
0       Liverpool  Paddy Power  2025-10-25T19:16:54Z    Brentford   
1       Liverpool  Paddy Power  2025-10-25T19:16:54Z    Liverpool   


In [4]:
#{(home_team, away_team): {outcome: [(odds, bookmaker), ...]}}
odds_by_match = {}

for _, row in df.iterrows():
    match_key = (row['home_team'], row['away_team'])
    outcome = row['outcome_name']
    odds = row['odds_decimal']
    bookmaker = row['bookmaker']
    
    if match_key not in odds_by_match:
        odds_by_match[match_key] = {}
    
    if outcome not in odds_by_match[match_key]:
        odds_by_match[match_key][outcome] = []
    
    odds_by_match[match_key][outcome].append((odds, bookmaker))

for match_key, outcomes in odds_by_match.items():
    print(f"\nMatch: {match_key[0]} vs {match_key[1]}")
    print("=" * 60)
    for outcome, odds_list in outcomes.items():
        print(f"  {outcome}:")
        for odds, bookmaker in odds_list:
            print(f"    {bookmaker}: {odds}")


Match: Brentford vs Liverpool
  Brentford:
    Paddy Power: 2.5
    Unibet (UK): 2.5
    Sky Bet: 2.6
  Liverpool:
    Paddy Power: 2.62
    Unibet (UK): 2.7
    Sky Bet: 2.6
  Draw:
    Paddy Power: 3.25
    Unibet (UK): 3.6
    Sky Bet: 3.5

Match: Arsenal vs Crystal Palace
  Arsenal:
    Paddy Power: 1.36
    William Hill: 1.4
    Marathon Bet: 1.42
    Unibet (UK): 1.4
    Sky Bet: 1.36
  Crystal Palace:
    Paddy Power: 8.0
    William Hill: 7.5
    Marathon Bet: 8.2
    Unibet (UK): 8.0
    Sky Bet: 8.0
  Draw:
    Paddy Power: 4.5
    William Hill: 4.5
    Marathon Bet: 4.75
    Unibet (UK): 4.8
    Sky Bet: 4.75

Match: Aston Villa vs Manchester City
  Aston Villa:
    Paddy Power: 4.33
    William Hill: 4.2
    Marathon Bet: 4.3
    Unibet (UK): 4.25
    Sky Bet: 4.33
  Manchester City:
    Paddy Power: 1.75
    William Hill: 1.75
    Marathon Bet: 1.81
    Unibet (UK): 1.8
    Sky Bet: 1.75
  Draw:
    Paddy Power: 3.8
    William Hill: 3.8
    Marathon Bet: 3.98
    Unibet 

In [ ]:
import pprint

# accounts for slippage, ect
epsilon = 0.05

def calc_arbitrage(odds1, odds2, odds3):
    return (1/odds1) + (1/odds2) + (1/odds3)

def find_arbritrage(odds_by_match):
    oppourtunities = []
    for match_key, outcomes in odds_by_match.items():

        if len(outcomes) != 3:
            continue

        best_h_val = 1-epsilon
        current_best = None
    
        outcome1, outcome2, outcome3 = outcomes.keys()

        for odds1, bookie1 in outcomes[outcome1]:
            for odds2, bookie2 in outcomes[outcome2]:
                for odds3, bookie3 in outcomes[outcome3]:

                    if len({bookie1, bookie2, bookie3}) == 3:
                        h_val = calc_arbitrage(odds1, odds2, odds3)
                        if h_val < best_h_val:
                            best_h_val = h_val
                            current_best = {
                                outcome1: (odds1, bookie1),
                                outcome2: (odds2, bookie2),
                                outcome3: (odds3, bookie3)
                            }

        if current_best:
            profit_margin = ((1/best_h_val)-1)*100
            oppourtunities.append({
                'match': f"{match_key[0]} vs {match_key[1]}",
                'h_value': best_h_val,
                'profit_margin_%': profit_margin,
                'combination': current_best
            })
    
    return sorted(oppourtunities, key = lambda x : x["h_val"])
    
arb_opps = find_arbritrage(odds_by_match)
pprint(arb_opps)


TypeError: unhashable type: 'list'